# Гипотезы и их проверка

## Вспомогательные функции
### Импорты

In [5]:
import kagglehub
import os
import cv2
import numpy as np
from typing import List
from src.features import (
    GreenPixelDensity,
    StraightAngles,
    WhiteAndBluePixelDensity,
    ShadowAndTextureContrast,
    BlueAndTurquoiseGradientDensity,
    TextureSmoothness,
    HorizontalLineDensity,
    VerticalLineDensity,
    AsphaltAndStoneTextureDensity,
    OrganicShapesAndCurves, MountainEdgeDensity, WaterReflectionDensity
)
from scipy.stats import (
    ttest_ind,
    ks_2samp,
    mannwhitneyu, fisher_exact, f_oneway, chi2_contingency, kruskal, spearmanr, levene,
pearsonr
)

### Загрузка датасета

In [6]:
# Download latest version
path = os.path.join(kagglehub.dataset_download("rahmasleam/intel-image-dataset"), "Intel Image Dataset")

print("Путь к файлам датасета:", path)

Путь к файлам датасета: /Users/vladyoslav/.cache/kagglehub/datasets/rahmasleam/intel-image-dataset/versions/1/Intel Image Dataset


### Функция загрузки изображений из директории

In [7]:
def load_images_from_folder(folder: str) -> List[np.ndarray]:
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            images.append(img)
    return images

### Получение всех категорий из датасета

In [8]:
categories = list(filter(lambda category: category != '.DS_Store', os.listdir(path)))

print(categories)

['forest', 'buildings', 'glacier', 'street', 'mountain', 'sea']


### Функция загрузки изображений всех категорий, кроме указанной

In [9]:
def load_other_categories_images(exclude_category: str) -> List[np.ndarray]:
    images = []

    for category in categories:
        if category != exclude_category:
            category_path = os.path.join(path, category)
            images.extend(load_images_from_folder(category_path))

    return images

### Функция вывода результатов проверки

In [10]:
# Significance level for p-value
ALPHA = 0.05

def print_test_results(test_results: List[tuple[str, tuple]], success_message: str, failure_message: str) -> None:
    for test_name, stat, p_value in test_results:
        print(f"{test_name}:")
        print(f"Statistic: {stat}")
        print(f"P-value: {p_value}\n")

    if all([p_value < ALPHA for _, _, p_value in test_results]):
        print(f"Гипотеза подтверждена: {success_message}")
    else:
        print(f"Гипотеза не подтверждена: {failure_message}")

## Проверка гипотез

### Гипотеза 1: Изображения зданий имеют больше прямых углов и повторяющихся геометрических форм.

В данной гипотезе предполагается, что изображения зданий будут содержать больше прямых углов и повторяющихся геометрических форм по сравнению с изображениями других категорий. Это связано с характерной архитектурой зданий, которая часто включает четкие линии и углы, в отличие от природы или других объектов.

### Способы проверки:

1. **T-тест**:
   Этот тест используется для сравнения средних значений признаков (количества прямых углов и повторяющихся форм) между изображениями зданий и изображениями других категорий. Если p-value меньше 0.05, это будет означать, что средние значения для двух групп статистически различны.

2. **KS тест (Колмогорова-Смирнова)**:
   Этот тест применяется для сравнения распределений признаков между двумя группами изображений. Он проверяет, есть ли статистически значимые различия в распределении признаков между изображениями зданий и изображениями других категорий.

3. **Тест Манна-Уитни U**:
   Этот тест используется для сравнения медианных значений между двумя группами изображений, чтобы понять, есть ли различия в распределении признаков для зданий и других категорий. Он применим, когда данные не обязательно имеют нормальное распределение.

In [11]:
# Instantiate the feature extractor
feature_extractor = StraightAngles()

category = "buildings"
buildings_path = os.path.join(path, category)

# Load images from directories
buildings_images = load_images_from_folder(buildings_path)
other_images = load_other_categories_images(exclude_category=category)

# Calculate features for both sets of images
buildings_features = [feature_extractor.calculate(img) for img in buildings_images]
other_features = [feature_extractor.calculate(img) for img in other_images]

t_stat, p_value_ttest = ttest_ind(buildings_features, other_features, equal_var=False)
ks_stat, p_value_ks = ks_2samp(buildings_features, other_features)
u_stat, p_value_u = mannwhitneyu(buildings_features, other_features, alternative='two-sided')

# Collect test results
test_results = [
    ("T-тест", t_stat, p_value_ttest),
    ("KS тест", ks_stat, p_value_ks),
    ("Критерий Манна-Уитни U", u_stat, p_value_u)
]

# Print results with conclusion
print_test_results(
    test_results,
    success_message="Здания имеют больше прямых углов и повторяющихся форм.",
    failure_message="Значимой разницы не обнаружено."
)


T-тест:
Statistic: -16.51083226267886
P-value: 1.509928324947185e-50

KS тест:
Statistic: 0.39113024550213343
P-value: 9.967623826567296e-52

Критерий Манна-Уитни U:
Statistic: 286184.0
P-value: 3.596143379458356e-60

Гипотеза подтверждена: Здания имеют больше прямых углов и повторяющихся форм.


### Гипотеза 2: Изображения леса имеют больше зелёных пикселей по сравнению с другими категориями.

В данной гипотезе предполагается, что изображения леса содержат больше зелёных пикселей, чем изображения других категорий. Это основано на том, что леса обычно характеризуются преобладанием зеленых оттенков, связанных с растительностью. Мы будем измерять долю пикселей с преобладанием зеленого канала, чтобы подтвердить или опровергнуть гипотезу.

### Способы проверки:

1. **T-тест**:
   Этот тест используется для сравнения средних значений доли зелёных пикселей между изображениями леса и изображениями других категорий. Если p-value меньше 0.05, это будет означать, что средние значения для двух групп статистически различны.

2. **ANOVA (F-статистика)**: Проверяет, есть ли различия между средними значениями более чем двух групп. Может быть полезным, если бы мы рассматривали несколько категорий помимо леса.

3. **Тест Фишера**: Этот тест используется для анализа категориальных данных. Он оценивает связь между двумя переменными, особенно полезен, если данные представляют собой категории (например, если мы категоризируем изображения как "больше зелёных пикселей" или "меньше").

In [12]:
# Instantiate the feature extractor
feature_extractor = GreenPixelDensity()

category = "forest"
forest_path = os.path.join(path, category)

# Load images from directories
forest_images = load_images_from_folder(forest_path)
other_images = load_other_categories_images(exclude_category=category)

# Calculate features for both sets of images
forest_features = [feature_extractor.calculate(img) for img in forest_images]
other_features = [feature_extractor.calculate(img) for img in other_images]

# Perform statistical tests
t_stat, p_value_ttest = ttest_ind(forest_features, other_features, equal_var=False)  # T-test
f_stat, p_value_f = f_oneway(forest_features, other_features)  # ANOVA test

# Fisher's Exact Test - This test requires a contingency table
# Calculate a contingency table based on a threshold for green pixels
threshold = 0.1  # Assume an image with more than 10% green pixels is considered "high green"
forest_high_green = np.sum(np.array(forest_features) > threshold)
forest_low_green = len(forest_features) - forest_high_green
other_high_green = np.sum(np.array(other_features) > threshold)
other_low_green = len(other_features) - other_high_green

# Contingency table: [high green, low green] for both forest and other categories
contingency_table = np.array([
    [forest_high_green, forest_low_green],
    [other_high_green, other_low_green]
])

# Perform Fisher's Exact Test
_, p_value_fisher = fisher_exact(contingency_table)


# Collect test results
test_results = [
    ("T-тест", t_stat, p_value_ttest),
    ("ANOVA тест", f_stat, p_value_f),
    ("Тест Фишера", None, p_value_fisher)  # Fisher's test doesn't return stat but p-value
]

# Print results with conclusion
print_test_results(
    test_results,
    success_message="Изображения леса имеют больше зелёных пикселей.",
    failure_message="Значимой разницы не обнаружено."
)


T-тест:
Statistic: 19.368645488388232
P-value: 3.069541337873858e-62

ANOVA тест:
Statistic: 1592.193376253794
P-value: 1.1997829253042167e-279

Тест Фишера:
Statistic: None
P-value: 5.69541257759806e-142

Гипотеза подтверждена: Изображения леса имеют больше зелёных пикселей.


### Гипотеза 3: На изображениях с ледниками преобладают оттенки белого и голубого цветов.

В данной гипотезе предполагается, что изображения ледников содержат больше пикселей белого и голубого цветов по сравнению с изображениями других категорий. Это связано с характерными свойствами льда и снега, которые отражают свет, создавая белые области, а преломление света внутри льда формирует голубые оттенки. Мы будем измерять долю пикселей, попадающих в диапазоны белого и голубого цветов, чтобы подтвердить или опровергнуть гипотезу.

### Способы проверки:

1. **T-тест**:
   Используется для сравнения средних значений доли бело-голубых пикселей между изображениями ледников и изображениями других категорий. Если p-value меньше 0.05, это укажет на наличие статистически значимых различий.

2. **ANOVA (F-статистика)**:
   Применяется для проверки различий между средними значениями доли бело-голубых пикселей в нескольких категориях изображений. Этот метод позволяет выявить, есть ли значимые различия между группами.

3. **Тест Манна-Уитни U**:
   Непараметрический тест для оценки различий в распределении доли бело-голубых пикселей между двумя группами изображений, что особенно полезно при наличии асимметричных распределений.

In [13]:
# Instantiate the feature extractor
feature_extractor = WhiteAndBluePixelDensity()

category = "glacier"
glacier_path = os.path.join(path, category)

# Load images from directories
glacier_images = load_images_from_folder(glacier_path)
other_images = load_other_categories_images(exclude_category=category)

# Calculate features for both sets of images
glacier_features = [feature_extractor.calculate(img) for img in glacier_images]
other_features = [feature_extractor.calculate(img) for img in other_images]

# Perform statistical tests
t_stat, p_value_ttest = ttest_ind(glacier_features, other_features, equal_var=False)
f_stat, p_value_anova = f_oneway(glacier_features, other_features)
u_stat, p_value_u = mannwhitneyu(glacier_features, other_features, alternative='two-sided')

# Collect test results
test_results = [
    ("T-test", t_stat, p_value_ttest),
    ("ANOVA", f_stat, p_value_anova),
    ("Mann-Whitney U", u_stat, p_value_u)
]

# Print results with conclusion
print_test_results(
    test_results,
    success_message="Изображения с ледниками имеют больше белых и голубых оттенков.",
    failure_message="Значимой разницы не обнаружено."
)


T-test:
Statistic: 24.952377351597207
P-value: 6.13522424199588e-103

ANOVA:
Statistic: 611.2198938855358
P-value: 5.706889749371772e-123

Mann-Whitney U:
Statistic: 1072602.5
P-value: 8.727437017509387e-103

Гипотеза подтверждена: Изображения с ледниками имеют больше белых и голубых оттенков.


### Гипотеза 4: Горы чаще всего содержат больше теней и текстур с высокими контрастами.

В данной гипотезе предполагается, что изображения гор характеризуются большей степенью теней и текстур с высокими контрастами, чем изображения других категорий. Это связано с природой горного ландшафта, где присутствуют резкие перепады высот, сложные текстуры и игра света и тени. Для проверки гипотезы будет рассчитан показатель контраста текстур и теней с использованием преобразования Лапласа.

### Способы проверки:

1. **Тест Колмогорова-Смирнова**:
   Используется для проверки сходства распределений уровня контраста между изображениями гор и изображениями других категорий. Если p-value меньше 0.05, распределения будут считаться различными.

2. **Критерий Хи-квадрат**:
   Применяется для проверки связи между категорией изображения (горы или другие категории) и наличием высокого уровня контраста. Это особенно полезно при анализе категориальных данных.

3. **ANOVA (F-статистика)**:
   Проверяет различия между средними уровнями контраста в нескольких категориях изображений.

In [14]:
# Instantiate the feature extractor
feature_extractor = ShadowAndTextureContrast()

category = "mountain"
mountain_path = os.path.join(path, category)

# Load images from directories
mountain_images = load_images_from_folder(mountain_path)
other_images = load_other_categories_images(exclude_category=category)

# Calculate features for both sets of images
mountain_features = [feature_extractor.calculate(img) for img in mountain_images]
other_features = [feature_extractor.calculate(img) for img in other_images]

# Perform statistical tests
ks_stat, p_value_ks = ks_2samp(mountain_features, other_features)
chi2_table = [
    [
        sum(1 for x in mountain_features if x > 0.5),
        sum(1 for x in mountain_features if x <= 0.5)
    ],
    [
        sum(1 for x in other_features if x > 0.5),
        sum(1 for x in other_features if x <= 0.5)
    ]
]

chi2_stat, p_value_chi2, _, _ = chi2_contingency(chi2_table)
f_stat, p_value_anova = f_oneway(mountain_features, other_features)

# Collect test results
test_results = [
    ("Kolmogorov-Smirnov Test", ks_stat, p_value_ks),
    ("Chi-Square Test", chi2_stat, p_value_chi2),
    ("ANOVA", f_stat, p_value_anova)
]

# Print results with conclusion
print_test_results(
    test_results,
    success_message="Изображения гор имеют больше теней и контрастов.",
    failure_message="Значимой разницы не обнаружено."
)


Kolmogorov-Smirnov Test:
Statistic: 0.22637806637806637
P-value: 5.862583276234776e-20

Chi-Square Test:
Statistic: 34.59137275361692
P-value: 4.067048031615426e-09

ANOVA:
Statistic: 129.50293413130308
P-value: 2.093525245601298e-29

Гипотеза подтверждена: Изображения гор имеют больше теней и контрастов.


### Гипотеза 5: На изображениях моря преобладают синие и бирюзовые цвета с плавными градиентами.

В данной гипотезе предполагается, что изображения моря имеют преобладание синих и бирюзовых оттенков с плавными градиентами. Это основано на том, что изображения морского ландшафта часто содержат такие цвета, как синий и бирюзовый, а также плавные переходы между цветами воды и неба. Мы будем измерять плотность этих оттенков и оценивать плавность градиентов, чтобы подтвердить или опровергнуть гипотезу.

### Способы проверки:

1. **Mann-Whitney U**:
   Этот тест используется для оценки различий между двумя независимыми выборками, в данном случае между изображениями моря и изображениями других категорий. Если p-value меньше 0.05, это будет означать, что данные группы статистически различны.

2. **Kruskal-Wallis H-test**:
   Этот тест проверяет, есть ли статистически значимые различия между более чем двумя независимыми группами. Мы используем его для оценки различий между изображениями моря и других категорий по плотности синих и бирюзовых оттенков.

3. **T-тест**:
   Этот тест применяется для сравнения средних значений между двумя группами. Мы проверим среднюю плотность оттенков синего и бирюзового цвета между изображениями моря и других категорий.

In [15]:
# Instantiate the feature extractor
feature_extractor = BlueAndTurquoiseGradientDensity()

category = "street"
sea_path = os.path.join(path, category)

# Load images from directories
sea_images = load_images_from_folder(sea_path)
other_images = load_other_categories_images(exclude_category=category)

# Calculate features for both sets of images
sea_features = [feature_extractor.calculate(img) for img in sea_images]
other_features = [feature_extractor.calculate(img) for img in other_images]

# Perform statistical tests
t_stat, p_value_ttest = ttest_ind(sea_features, other_features, equal_var=False)
h_stat, p_value_kruskal = kruskal(sea_features, other_features)
u_stat, p_value_u = mannwhitneyu(sea_features, other_features, alternative='two-sided')

# Collect test results
test_results = [
    ("T-test", t_stat, p_value_ttest),
    ("Kruskal-Wallis", h_stat, p_value_kruskal),
    ("Mann-Whitney U", u_stat, p_value_u)
]

# Print results with conclusion
print_test_results(
    test_results,
    success_message="Изображения моря имеют больше синих и бирюзовых оттенков с плавными градиентами.",
    failure_message="Значимой разницы не обнаружено."
)

T-test:
Statistic: -19.343418163532057
P-value: 2.1572479864574292e-78

Kruskal-Wallis:
Statistic: 177.41395622569124
P-value: 1.778609986343315e-40

Mann-Whitney U:
Statistic: 390696.0
P-value: 1.7792843689068248e-40

Гипотеза подтверждена: Изображения моря имеют больше синих и бирюзовых оттенков с плавными градиентами.


### Гипотеза 6: На изображениях ледников текстуры преимущественно гладкие и однородные.

Данная гипотеза предполагает, что изображения ледников будут характеризоваться гладкими и однородными текстурами, что связано с особенностями ледников, где поверхности часто представляют собой равномерно освещенные и монотонные области без ярко выраженных деталей или изменений. Мы будем измерять стандартное отклонение интенсивности пикселей в изображении, чтобы подтвердить или опровергнуть гипотезу.

### Способы проверки:

1. **Mann-Whitney U**: Этот тест используется для сравнения двух независимых выборок. Он оценивает, отличаются ли распределения значений между двумя группами (например, ледники и другие изображения). Если p-value меньше 0.05, это указывает на статистически значимые различия между группами.

2. **Spearman's rank correlation**: Оценивает степень монотонной зависимости между двумя переменными. Этот критерий помогает понять, существует ли зависимость между гладкостью текстуры и наличием ледников.

3. **Kruskal-Wallis**: Этот тест является аналогом ANOVA, но используется для проверки различий между несколькими независимыми группами. Подходит, если мы хотим сравнить более двух категорий, например, ледники и другие природные объекты.

In [16]:
# Instantiate the feature extractor
feature_extractor = TextureSmoothness()

category = "glacier"
glacier_path = os.path.join(path, category)

# Load images from directories
glacier_images = load_images_from_folder(glacier_path)
other_images = load_other_categories_images(exclude_category=category)

# Calculate features for both sets of images
glacier_features = [feature_extractor.calculate(img) for img in glacier_images]
other_features = [feature_extractor.calculate(img) for img in other_images]

# Perform statistical tests
mannwhitney_stat, p_value_mannwhitney = mannwhitneyu(glacier_features, other_features, alternative='two-sided')
spearman_stat, p_value_spearman = spearmanr(glacier_features, other_features[:len(glacier_features)])
kruskal_stat, p_value_kruskal = kruskal(glacier_features, other_features)

# Collect test results
test_results = [
    ("Mann-Whitney U", mannwhitney_stat, p_value_mannwhitney),
    ("Spearman's rank correlation", spearman_stat, p_value_spearman),
    ("Kruskal-Wallis", kruskal_stat, p_value_kruskal)
]

# Print results with conclusion
print_test_results(
    test_results,
    success_message="Изображения ледников имеют гладкие и однородные текстуры.",
    failure_message="Значимой разницы не обнаружено."
)


Mann-Whitney U:
Statistic: 603246.0
P-value: 6.68466685161619e-05

Spearman's rank correlation:
Statistic: 0.058744088763420826
P-value: 0.1677440836512504

Kruskal-Wallis:
Statistic: 15.898291859153687
P-value: 6.68390135819863e-05

Гипотеза не подтверждена: Значимой разницы не обнаружено.


### Гипотеза 7: Морские изображения часто содержат горизонтальные линии, связанные с горизонтом

Данная гипотеза предполагает, что на изображениях моря часто встречаются горизонтальные линии, такие как линия горизонта. Мы будем измерять плотность горизонтальных линий на изображениях, чтобы проверить, поддерживает ли гипотеза предположение о наличии этих линий.

### Способы проверки:

1. **T-test**: Используется для проверки гипотезы о том, что изображения моря имеют среднюю плотность горизонтальных линий, отличную от других категорий. Если p-value меньше 0.05, это будет означать значимые различия между группами.

2. **Levene's Test**: Этот тест проверяет равенство дисперсий плотности горизонтальных линий между изображениями моря и других категорий. Если p-value меньше 0.05, это указывает на статистически значимые различия в дисперсиях.

3. **Chi-square Test**: Этот тест применим для проверки, есть ли зависимость между наличием горизонтальных линий и категорией изображения. Если p-value меньше 0.05, гипотеза будет подтверждена.

In [17]:
# Initialize the feature extractor
feature_extractor = HorizontalLineDensity()

category = "sea"
sea_path = os.path.join(path, category)

# Load images from directories
sea_images = load_images_from_folder(sea_path)
other_images = load_other_categories_images(exclude_category=category)

# Calculate features for both sets of images
sea_features = [feature_extractor.calculate(img) for img in sea_images]
other_features = [feature_extractor.calculate(img) for img in other_images]

# Split the data into bins for histogram
bins = 10
hist_sea, bin_edges_sea = np.histogram(sea_features, bins=bins)
hist_other, bin_edges_other = np.histogram(other_features, bins=bins)

# Combine the histograms into a frequency table
frequency_table = np.array([hist_sea, hist_other])

# Check: if there are any zeros in the table, replace them with the minimum value (e.g., 1)
frequency_table[frequency_table == 0] = 1

# Perform statistical tests
t_stat, p_value_ttest = ttest_ind(sea_features, other_features, equal_var=False)
levene_stat, p_value_levene = levene(sea_features, other_features)
chi2_stat, p_value_chi2, dof, expected = chi2_contingency(frequency_table)

# Collect the test results
test_results = [
    ("T-test", t_stat, p_value_ttest),
    ("Levene's Test", levene_stat, p_value_levene),
    ("Chi-square Test", chi2_stat, p_value_chi2)
]

# Print results with conclusion in Russian
print_test_results(
    test_results,
    success_message="Изображения моря содержат больше горизонтальных линий, связанных с горизонтом.",
    failure_message="Значимой разницы не обнаружено."
)


T-test:
Statistic: -5.205812628417344
P-value: 2.1510515426320074e-07

Levene's Test:
Statistic: 10.158613317880024
P-value: 0.001451058177210804

Chi-square Test:
Statistic: 13.456460444442678
P-value: 0.14301602082841786

Гипотеза не подтверждена: Значимой разницы не обнаружено.


### Гипотеза 8: Городские сцены содержат больше вертикальных линий и углов, связанных со зданиями и инфраструктурой.

В данной гипотезе предполагается, что изображения городских сцен содержат больше вертикальных линий, чем изображения других типов. Вертикальные линии часто ассоциируются с городской архитектурой, зданиями и инфраструктурой. Мы будем измерять плотность вертикальных линий, чтобы подтвердить или опровергнуть гипотезу.

### Способы проверки:

1. **T-тест**:
   Этот тест используется для сравнения средних значений плотности вертикальных линий между изображениями городской сцены и изображениями других категорий. Если p-value меньше 0.05, это будет означать, что средние значения для двух групп статистически различны.

2. **Критерий Манна-Уитни (Mann-Whitney U)**:
   Применяется для проверки различий в распределении плотности вертикальных линий между изображениями городской сцены и изображениями других категорий, особенно если данные не следуют нормальному распределению.

3. **Тест Крускала (Kruskal-Wallis)**:
   Этот тест используется для проверки различий в распределении плотности вертикальных линий между несколькими категориями изображений. Он поможет определить, есть ли различия в плотности вертикальных линий между изображениями городской сцены и другими типами изображений.


In [18]:
# Instantiate the feature extractor
feature_extractor = VerticalLineDensity()

category = "street"
city_path = os.path.join(path, category)

# Load images from directories
city_images = load_images_from_folder(city_path)
other_images = load_other_categories_images(exclude_category=category)

# Calculate features for both sets of images
city_features = [feature_extractor.calculate(img) for img in city_images]
other_features = [feature_extractor.calculate(img) for img in other_images]

# Perform statistical tests
t_stat, p_value_ttest = ttest_ind(city_features, other_features, equal_var=False)
u_stat, p_value_u = mannwhitneyu(city_features, other_features, alternative='two-sided')
h_stat, p_value_kruskal = kruskal(city_features, other_features)

# Collect test results
test_results = [
    ("T-test", t_stat, p_value_ttest),
    ("Mann-Whitney U", u_stat, p_value_u),
    ("Kruskal-Wallis", h_stat, p_value_kruskal)
]

# Print results with conclusion
print_test_results(
    test_results,
    success_message="Изображения городской сцены содержат больше вертикальных линий.",
    failure_message="Значимой разницы не обнаружено."
)


T-test:
Statistic: -3.959298621113764
P-value: 8.121208221938546e-05

Mann-Whitney U:
Statistic: 596987.5
P-value: 0.004713037134223194

Kruskal-Wallis:
Statistic: 7.986662402168019
P-value: 0.004712320556597948

Гипотеза подтверждена: Изображения городской сцены содержат больше вертикальных линий.


### Гипотеза 9: Уличные сцены содержат больше текстур асфальта и камня, чем природные категории.

Данная гипотеза предполагает, что уличные сцены будут содержать больше текстур, характерных для асфальта и камня, по сравнению с природными категориями. Это основано на том, что асфальт и камень имеют характерные текстурные паттерны, которые можно обнаружить в изображениях городских сцен. Мы будем измерять плотность текстур в изображениях уличных сцен и других категорий, чтобы подтвердить или опровергнуть гипотезу.

### Способы проверки:

1. **Mann-Whitney U**:
   Этот тест используется для проверки различий в распределении плотности текстур между уличными сценами и природными изображениями, если данные не имеют нормального распределения.

2. **Spearman's rank correlation**: Проверяет, существует ли монотонная зависимость между плотностью текстур и категориальной переменной, которая различает уличные и природные сцены.

3. **Kruskal-Wallis**: Этот тест используется для проверки различий в плотности текстур между более чем двумя категориями, если в наборе данных есть несколько категорий помимо уличных сцен.


In [19]:
# Instantiate the feature extractor
feature_extractor = AsphaltAndStoneTextureDensity()

category = "street"
street_path = os.path.join(path, category)

# Load images from directories
street_images = load_images_from_folder(street_path)
other_images = load_other_categories_images(exclude_category=category)

# Ensure both categories have the same number of images for comparison
min_length = min(len(street_images), len(other_images))
street_images = street_images[:min_length]
other_images = other_images[:min_length]

# Calculate features for both sets of images
street_features = [feature_extractor.calculate(img) for img in street_images]
other_features = [feature_extractor.calculate(img) for img in other_images]

# Perform statistical tests
t_stat, p_value_ttest = ttest_ind(street_features, other_features, equal_var=False)
pearson_stat, p_value_pearson = pearsonr(street_features, other_features)

# Prepare the data for chi2_contingency
bin_count = 10  # Choose the number of bins based on the distribution of feature values
street_bins = np.histogram(street_features, bins=bin_count)[0]
other_bins = np.histogram(other_features, bins=bin_count)[0]

# Create the contingency table
contingency_table = np.array([street_bins, other_bins])

# Perform Chi-Square Test
chi2_stat, p_value_chi2, _, _ = chi2_contingency(contingency_table)

# Collect test results
test_results = [
    ("T-test", t_stat, p_value_ttest),
    ("Pearson's correlation", pearson_stat, p_value_pearson),
    ("Chi-square Test", chi2_stat, p_value_chi2)
]

# Print results with conclusion
print_test_results(
    test_results,
    success_message="Уличные сцены содержат больше текстур асфальта и камня, чем природные категории.",
    failure_message="Значимой разницы не обнаружено."
)

T-test:
Statistic: -23.367171193616773
P-value: 1.3624125141205788e-96

Pearson's correlation:
Statistic: -0.021607481997517435
P-value: 0.6294584976101333

Chi-square Test:
Statistic: 172.15950643944234
P-value: 2.1895631497677404e-32

Гипотеза не подтверждена: Значимой разницы не обнаружено.


### Гипотеза 10: Изображения лесов имеют больше органических форм и кривых, что связано с природным ростом деревьев и растений.

Данная гипотеза предполагает, что лесные изображения будут содержать больше органических форм и кривых, которые характерны для природного роста деревьев и растений, по сравнению с другими категориями. Это основано на том, что природные формы, такие как ветви деревьев, листья и другие элементы растительности, имеют извилистые, непрямые контуры, в отличие от прямых линий в урбанистических сценах. Мы будем измерять количество кривых и органических контуров в изображениях лесов и других категорий, чтобы подтвердить или опровергнуть гипотезу.

### Способы проверки:

1. **Kruskal-Wallis**: Этот тест используется для проверки различий в плотности органических форм между лесными изображениями и изображениями других категорий, если данные не имеют нормального распределения.

2. **Spearman's rank correlation**: Проверяет, существует ли монотонная зависимость между количеством органических форм и категориальной переменной, которая различает лесные и другие сцены.

3. **Levene's test**: Этот тест используется для проверки равенства дисперсий в группах органических форм между лесными изображениями и изображениями других категорий.


In [20]:
# Instantiate the feature extractor
feature_extractor = OrganicShapesAndCurves()

category = "forest"
forest_path = os.path.join(path, category)

# Load images from directories
forest_images = load_images_from_folder(forest_path)
other_images = load_other_categories_images(exclude_category=category)

# Calculate features for both sets of images
forest_features = [feature_extractor.calculate(img) for img in forest_images]
other_features = [feature_extractor.calculate(img) for img in other_images]

# Perform statistical tests
kruskal_stat, p_value_kruskal = kruskal(forest_features, other_features)
spearman_stat, p_value_spearman = spearmanr(forest_features, other_features[:len(forest_features)])
levene_stat, p_value_levene = levene(forest_features, other_features)

# Collect test results
test_results = [
    ("Kruskal-Wallis", kruskal_stat, p_value_kruskal),
    ("Spearman's rank correlation", spearman_stat, p_value_spearman),
    ("Levene's Test", levene_stat, p_value_levene)
]

# Print results with conclusion
print_test_results(
    test_results,
    success_message="Изображения лесов содержат больше органических форм и кривых.",
    failure_message="Значимой разницы не обнаружено."
)


Kruskal-Wallis:
Statistic: 357.5292978317767
P-value: 9.718009709303981e-80

Spearman's rank correlation:
Statistic: -0.032993423693577086
P-value: 0.4736109486342235

Levene's Test:
Statistic: 45.5928062144448
P-value: 1.7412477406752928e-11

Гипотеза не подтверждена: Значимой разницы не обнаружено.


### Гипотеза 11: В горных пейзажах часто присутствуют резкие и отчетливые линии, разделяющие небо и скалы.

Данная гипотеза предполагает, что изображения горных пейзажей будут содержать четкие границы или линии, разделяющие небо и скалы. Это связано с характерной структурой горных ландшафтов, где небо часто резко отделяется от земли или скалистых объектов. Мы будем измерять плотность границ на изображениях горных пейзажей и других категорий, чтобы подтвердить или опровергнуть гипотезу.

### Способы проверки:

1. **Mann-Whitney U**:
   Этот тест используется для проверки различий в распределении плотности границ между горными пейзажами и другими категориями, если данные не имеют нормального распределения.

2. **Spearman's rank correlation**: Проверяет, существует ли монотонная зависимость между плотностью границ и категориальной переменной, которая различает горные пейзажи и другие типы сцен.

3. **Kruskal-Wallis**: Этот тест используется для проверки различий в плотности границ между более чем двумя категориями, если в наборе данных есть несколько категорий помимо горных пейзажей.

In [21]:
# Instantiate the feature extractor
feature_extractor = MountainEdgeDensity()

category = "mountain"
mountain_path = os.path.join(path, category)

# Load images from directories
mountain_images = load_images_from_folder(mountain_path)
other_images = load_other_categories_images(exclude_category=category)

# Calculate features for both sets of images
mountain_features = [feature_extractor.calculate(img) for img in mountain_images]
other_features = [feature_extractor.calculate(img) for img in other_images]

# Perform statistical tests
mannwhitney_stat, p_value_mannwhitney = mannwhitneyu(mountain_features, other_features, alternative='two-sided')
spearman_stat, p_value_spearman = spearmanr(mountain_features, other_features[:len(mountain_features)])
kruskal_stat, p_value_kruskal = kruskal(mountain_features, other_features)

# Collect test results
test_results = [
    ("Mann-Whitney U", mannwhitney_stat, p_value_mannwhitney),
    ("Spearman's rank correlation", spearman_stat, p_value_spearman),
    ("Kruskal-Wallis", kruskal_stat, p_value_kruskal)
]

# Print results with conclusion
print_test_results(
    test_results,
    success_message="Изображения горных пейзажей содержат больше резких и отчетливых линий, разделяющих небо и скалы.",
    failure_message="Значимой разницы не обнаружено."
)


Mann-Whitney U:
Statistic: 315339.0
P-value: 8.518198686928201e-77

Spearman's rank correlation:
Statistic: -0.0933734984720385
P-value: 0.032433150312554

Kruskal-Wallis:
Statistic: 344.0166815459761
P-value: 8.513804900776157e-77

Гипотеза подтверждена: Изображения горных пейзажей содержат больше резких и отчетливых линий, разделяющих небо и скалы.


### Гипотеза 12: На изображениях моря часто можно заметить отражения от воды.

Данная гипотеза предполагает, что изображения моря будут иметь больше отражений от воды по сравнению с изображениями других категорий. Это основано на том, что вода может отражать объекты на поверхности, создавая характерные визуальные эффекты. Мы будем измерять плотность этих отражений на изображениях морских сцен и других категорий, чтобы подтвердить или опровергнуть гипотезу.

### Способы проверки:

1. **T-test**:
   Этот тест используется для проверки различий в плотности отражений между изображениями моря и других категорий, если данные имеют нормальное распределение.

2. **Levene's Test**: Проверяет, есть ли статистически значимые различия в вариативности плотности отражений между морем и другими категориями.

3. **Mann-Whitney U Test**: Этот тест используется для проверки различий в плотности отражений между изображениями моря и других категорий, если данные не имеют нормального распределения.


In [22]:
# Instantiate the feature extractor
feature_extractor = WaterReflectionDensity()

category = "sea"
sea_path = os.path.join(path, category)

# Load images from directories
sea_images = load_images_from_folder(sea_path)
other_images = load_other_categories_images(exclude_category=category)

# Calculate features for both sets of images
sea_features = [feature_extractor.calculate(img) for img in sea_images]
other_features = [feature_extractor.calculate(img) for img in other_images]

# Perform statistical tests
t_stat, p_value_ttest = ttest_ind(sea_features, other_features, equal_var=False)
levene_stat, p_value_levene = levene(sea_features, other_features)
u_stat, p_value_u = mannwhitneyu(sea_features, other_features, alternative='two-sided')

# Collect test results
test_results = [
    ("T-test", t_stat, p_value_ttest),
    ("Levene's Test", levene_stat, p_value_levene),
    ("Mann-Whitney U", u_stat, p_value_u)
]

# Print results with conclusion
print_test_results(
    test_results,
    success_message="Изображения моря содержат больше отражений от воды.",
    failure_message="Значимой разницы не обнаружено."
)


T-test:
Statistic: -13.726228281615025
P-value: 1.326617300900589e-39

Levene's Test:
Statistic: 65.49267201607903
P-value: 8.388185065709116e-16

Mann-Whitney U:
Statistic: 443412.5
P-value: 6.053186755437936e-27

Гипотеза подтверждена: Изображения моря содержат больше отражений от воды.


## Выводы

### Подтвержденные гипотезы:
- **Гипотеза 1**: Здания имеют больше прямых углов и повторяющихся форм.
- **Гипотеза 2**: Изображения леса имеют больше зелёных пикселей.
- **Гипотеза 3**: Изображения с ледниками имеют больше белых и голубых оттенков.
- **Гипотеза 4**: Изображения гор имеют больше теней и контрастов.
- **Гипотеза 5**: Изображения моря имеют больше синих и бирюзовых оттенков с плавными градиентами.
- **Гипотеза 8**: Изображения городской сцены содержат больше вертикальных линий.
- **Гипотеза 11**: Изображения горных пейзажей содержат больше резких и отчетливых линий, разделяющих небо и скалы.
- **Гипотеза 12**: Изображения моря содержат больше отражений от воды.

### Опровергнутые гипотезы:
- **Гипотеза 6**: Изображения ледников имеют гладкие и однородные текстуры. (Значимой разницы не обнаружено.)
- **Гипотеза 7**: Изображения моря содержат больше горизонтальных линий, связанных с горизонтом. (Значимой разницы не обнаружено.)
- **Гипотеза 9**: Уличные сцены содержат больше текстур асфальта и камня, чем природные категории. (Значимой разницы не обнаружено.)
- **Гипотеза 10**: Изображения лесов содержат больше органических форм и кривых. (Значимой разницы не обнаружено.)
